# RAG v2: Apply simatic chunking

In [1]:
# Basic setup
import os
from langchain_ollama import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

llm_model_name='qwen2.5:7b'
llm = ChatOllama(model=llm_model_name, temperature=0)

embedding_model='bge-m3:latest'
embeddings = OllamaEmbeddings(model=embedding_model)

In [5]:
# Helper functions
def replace_t_with_space(list_of_documents):
    """
    Thay thế tất cả các ký tự tab ('\t') bằng dấu cách trong nội dung trang của mỗi tài liệu

    Args:
        list_of_documents: Danh sách các đối tượng tài liệu, mỗi đối tượng có thuộc tính 'page_content'.

    Returns:
        Danh sách tài liệu đã được sửa đổi với các ký tự tab được thay thế bằng dấu cách.
    """

    for doc in list_of_documents:
        doc.page_content = doc.page_content.replace('\t', ' ')  # Replace tabs with spaces
    return list_of_documents
  


In [3]:
# Load pdf file and split file to chunks
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker

PDF_FILE_PATH='data/law_gtdb_35.pdf'
# chunk_size = 1000
# chunk_overlap = chunk_size // 5

loader = PyPDFLoader(PDF_FILE_PATH)
documents = loader.load()

text_splitter = SemanticChunker(
    # Initialize the semantic chunker using OllamaEmbeddings's embedding model
    OllamaEmbeddings(model=embedding_model),
    # Set the split breakpoint type to percentile
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=70,
)


docs = text_splitter.split_documents(documents)

In [4]:
docs[0]

Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2024-09-20T10:02:58+07:00', 'moddate': '2024-09-20T10:02:58+07:00', 'source': 'data/law_gtdb_35.pdf', 'total_pages': 69, 'page': 0, 'page_label': '1'}, page_content='CÔNG BÁO/Số 983 + 984/Ngày 25-8-2024 3 \n \n \nQUỐC HỘI \n \nCỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM \nĐộc lập - Tự do - Hạnh phúc \nLuật số: 35/2024/QH15  \n \nLUẬT \nĐƯỜNG BỘ \n \nCăn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam; \nQuốc hội ban hành Luật Đường bộ. Chương I \nNHỮNG QUY ĐỊNH CHUNG \n \nĐiều 1.')

In [6]:
cleaned_docs = replace_t_with_space(docs)

In [7]:
# Save to vector store
vectorstore = Chroma.from_documents(
  documents=cleaned_docs,
  collection_name='GTDB_35_v2',
  embedding=embeddings)

In [8]:
# create retrieve
retriever = vectorstore.as_retriever(
  search_type="similarity",
  search_kwargs={'k': 4}, # number of documents to retrieve
)

In [9]:
question = "Quy định đặt tên đường bộ là gì?"
retriever_docs = retriever.invoke(question)
retriever_docs

[Document(metadata={'creationdate': '2024-09-20T10:02:58+07:00', 'creator': 'PyPDF', 'moddate': '2024-09-20T10:02:58+07:00', 'page': 7, 'page_label': '8', 'producer': 'PyPDF', 'source': 'data/law_gtdb_35.pdf', 'total_pages': 69}, page_content='Đặt tên, đổi tên, số hiệu đường bộ \n1. Việc đặt tên, số hiệu đường bộ được quy định như sau: \na) Tên đường bộ được đặt theo tên danh nhân, người có công với đất nước; di \ntích, sự kiện lịch sử, văn hóa; tên địa danh hoặc tên theo tập quán. Số hiệu đường \nbộ được đặt theo số tự nhiên hoặc số tự nhiên kèm theo chữ cái nếu cần thiết. Trường hợp đường đô thị trùng với đường khác thì sử dụng cả tên đường đô thị và \ntên, số hiệu của đường khác; \nb) Tên, số hiệu đường bộ tham gia vào mạng lưới đường bộ quốc tế thực hiện \ntheo điều ước quốc tế giữa nước Cộng hòa xã hội chủ nghĩa Việt Nam với các quốc \ngia, tổ chức quốc tế có liên quan. Đường bộ kết nối vào mạng lưới đường bộ quốc tế \nthì sử dụng cả tên, số hiệu đường bộ trong nước và tên, số hiệ

In [10]:
retriever_docs[0].metadata

{'creationdate': '2024-09-20T10:02:58+07:00',
 'creator': 'PyPDF',
 'moddate': '2024-09-20T10:02:58+07:00',
 'page': 7,
 'page_label': '8',
 'producer': 'PyPDF',
 'source': 'data/law_gtdb_35.pdf',
 'total_pages': 69}

In [11]:
print(f"====== page: {retriever_docs[0].metadata['page_label']} ====== \n {retriever_docs[0].page_content}")

====== page: 8 ====== 
 Đặt tên, đổi tên, số hiệu đường bộ 
1. Việc đặt tên, số hiệu đường bộ được quy định như sau: 
a) Tên đường bộ được đặt theo tên danh nhân, người có công với đất nước; di 
tích, sự kiện lịch sử, văn hóa; tên địa danh hoặc tên theo tập quán. Số hiệu đường 
bộ được đặt theo số tự nhiên hoặc số tự nhiên kèm theo chữ cái nếu cần thiết. Trường hợp đường đô thị trùng với đường khác thì sử dụng cả tên đường đô thị và 
tên, số hiệu của đường khác; 
b) Tên, số hiệu đường bộ tham gia vào mạng lưới đường bộ quốc tế thực hiện 
theo điều ước quốc tế giữa nước Cộng hòa xã hội chủ nghĩa Việt Nam với các quốc 
gia, tổ chức quốc tế có liên quan. Đường bộ kết nối vào mạng lưới đường bộ quốc tế 
thì sử dụng cả tên, số hiệu đường bộ trong nước và tên, số hiệu đường bộ quốc tế. 2. Trường hợp có tuyến, đoạn tuyến đường bộ đi trùng nhau thì sử dụng tên, số 
hiệu đường bộ thuộc cấp quản lý cao hơn, trừ trường hợp quy định tại điểm a 
khoản 1 Điều này.


### Check document relevancy

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )
llm = ChatOllama(model=llm_model_name)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

In [13]:
# Filter out the non-relevant docs
docs_to_use = []
for doc in retriever_docs:
    print(doc.page_content[:100],'... \n', '-'*50)
    res = retrieval_grader.invoke({"question": question, "document": doc.page_content})
    print(res,'\n')
    if res.binary_score == 'yes':
        docs_to_use.append(doc)

Đặt tên, đổi tên, số hiệu đường bộ 
1. Việc đặt tên, số hiệu đường bộ được quy định như sau: 
a) Tên ... 
 --------------------------------------------------
binary_score='yes' 

3. Không bắt buộc đặt tên, số hiệu đường bộ đối với đường xã, đường thôn, 
đường nội bộ, đường chuyê ... 
 --------------------------------------------------
binary_score='no' 

Bộ trưởng Bộ Giao thông vận tải quy định trình tự, thủ tục chấp thuận thiết kế 
và trình tự, thủ tục ... 
 --------------------------------------------------
binary_score='no' 

Thực hiện thống nhất trên cơ sở phân công, phân cấp trách nhiệm, quyền 
hạn cụ thể, đồng thời có sự  ... 
 --------------------------------------------------
binary_score='no' 



In [15]:
print(f"====== page: {retriever_docs[2].metadata['page_label']} ====== \n {retriever_docs[2].page_content}")

====== page: 30 ====== 
 Bộ trưởng Bộ Giao thông vận tải quy định trình tự, thủ tục chấp thuận thiết kế 
và trình tự, thủ tục cấp phép thi công nút giao đối với đường quốc lộ đang khai thác; 
Ủy ban nhân dân cấp tỉnh quy định trình tự, thủ tục chấp thuận thiết kế và trình tự, 
thủ tục cấp phép thi công nút giao đối với đường địa phương đang khai thác. Điều 31.


### Generate Result

In [16]:
from langchain_core.output_parsers import StrOutputParser

# Prompt
system = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge. 
Use three-to-five sentences maximum and keep the answer concise."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question>"),
    ]
)

# LLM
llm = ChatOllama(model=llm_model_name, temperature=0)

# Post-processing
def format_docs(docs):
    return "\n".join(f"<doc{i+1}>:\nSource:{doc.metadata['source']}\nContent:{doc.page_content}\n</doc{i+1}>\n" for i, doc in enumerate(docs))

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"documents":format_docs(docs_to_use), "question": question})
generation

'Quy định đặt tên đường bộ theo văn bản là: Tên đường được đặt theo danh nhân, địa danh, sự kiện lịch sử, hoặc tập quán; số hiệu được dùng số tự nhiên hoặc kết hợp với chữ cái. Tên và số hiệu của các tuyến đường đô thị trùng nhau sẽ sử dụng tên, số hiệu cấp quản lý cao hơn.'

### Check for Halluciation

In [17]:
# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in 'generation' answer."""

    binary_score: str = Field(
        ...,
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

# LLM with function call
llm = ChatOllama(model=llm_model_name, temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
    Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n <facts>{documents}</facts> \n\n LLM generation: <generation>{generation}</generation>"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader

response = hallucination_grader.invoke({"documents": format_docs(docs_to_use), "generation": generation})
print(response)

binary_score='yes'
